In [ ]:
# ╔═══════════════════════════════════════════════════════════════╗
# ║   Readwise → Last 7 Days Export (Documents + All Notes)       ║
# ║   FIXED VERSION: Handles date formats & empty docs            ║
# ║   Fully working in Google Colab – November 2025               ║
# ╚═══════════════════════════════════════════════════════════════╝

# Step 1: Install nothing extra – requests is already in Colab
import requests
from datetime import datetime, timedelta, timezone
import json
import time
import pandas as pd
from IPython.display import display, Markdown

# Step 2: PUT YOUR READWISE TOKEN HERE
# Get it from https://readwise.io/access_token
TOKEN = "your-token-here"   # ←←← CHANGE THIS!

if TOKEN == "YOUR_TOKEN_HERE" or len(TOKEN) < 20:
    raise ValueError("Please paste your real Readwise token above!")

headers = {"Authorization": f"Token {TOKEN}"}

# Step 3: Calculate "last 7 days" in UTC (strict ISO for APIs)
days_back = 7
cutoff_dt = datetime.now(timezone.utc) - timedelta(days=days_back)
cutoff = cutoff_dt.strftime("%Y-%m-%dT%H:%M:%SZ")  # e.g., "2025-11-19T12:34:56Z" – strict ISO with Z
cutoff_no_z = cutoff_dt.strftime("%Y-%m-%dT%H:%M:%S")  # For classic API fallback
print(f"Fetching everything updated after: {cutoff} (last {days_back} days)")

# ──────────────────────────────
# Part A: Get all recent DOCUMENTS from Reader API (with fallback)
# ──────────────────────────────
print("\nFetching documents from Reader API (articles, emails, PDFs, etc.)...")
all_documents = []
next_cursor = None
use_date_filter = True

# First try with date filter
params = {"updatedAfter": cutoff, "pageCursor": next_cursor}
url = "https://readwise.io/api/v3/list/"
r = requests.get(url, headers=headers, params=params)
if r.status_code == 400 and "updatedAfter" in r.text:
    print("   → Date filter issue detected. Falling back to fetch all + Python filter (one-time only).")
    use_date_filter = False
    # Fetch without date filter (paginated)
    params = {"pageCursor": next_cursor}
    r = requests.get(url, headers=headers, params=params)

data = r.json() if r.status_code == 200 else {}
batch = data.get("results", [])

if use_date_filter or not use_date_filter:
    all_documents.extend(batch)
    print(f"   → Got {len(batch)} documents (total so far: {len(all_documents)})")

    # Paginate if needed
    while True:
        next_cursor = data.get("nextPageCursor")
        if not next_cursor:
            break
        params["pageCursor"] = next_cursor
        time.sleep(0.5)
        r = requests.get(url, headers=headers, params=params)
        if r.status_code != 200:
            print(f"Pagination error: {r.status_code}")
            break
        data = r.json()
        batch = data.get("results", [])
        all_documents.extend(batch)
        print(f"   → Got {len(batch)} more (total: {len(all_documents)})")

# If fallback used, filter in Python
if not use_date_filter:
    cutoff_dt_obj = cutoff_dt  # Reuse datetime obj
    all_documents = [
        doc for doc in all_documents
        if datetime.fromisoformat(doc.get("updated_at", "").replace("Z", "+00:00")) > cutoff_dt_obj
    ]
    print(f"   → Python filtered to {len(all_documents)} recent docs")

if r.status_code != 200:
    print(f"Reader API error {r.status_code}: {r.text}")

print(f"\nFinished! Found {len(all_documents)} documents touched in the last {days_back} days.\n")

# ──────────────────────────────
# Part B: Get all recent HIGHLIGHTS/NOTES from classic Readwise API
# ──────────────────────────────
print("Fetching recent highlights & inline notes from Readwise API...")
all_highlights = []
url = "https://readwise.io/api/v2/highlights/"
params = {
    "highlighted_at__gt": cutoff_no_z,  # Classic API likes no Z
    "page_size": 1000
}

while True:
    r = requests.get(url, headers=headers, params=params)
    if r.status_code != 200:
        print(f"Error {r.status_code}: {r.text}")
        break

    data = r.json()
    batch = data.get("results", [])
    all_highlights.extend(batch)
    print(f"   → Got {len(batch)} highlights (total so far: {len(all_highlights)})")

    if not data.get("next"):
        break
    url = data["next"]  # Follow pagination URL
    params = {}  # No extra params for next pages
    time.sleep(0.5)

# If date filter failed here too (rare), fallback filter
if len(all_highlights) > 1000:  # Sanity check for fallback trigger
    print("   → Applying Python filter to highlights...")
    all_highlights = [
        h for h in all_highlights
        if datetime.fromisoformat(h.get("highlighted_at", "").replace("Z", "+00:00")) > cutoff_dt
    ]

print(f"Found {len(all_highlights)} highlights/notes made in the last {days_back} days.\n")

# ──────────────────────────────
# Part C: Combine everything nicely
# ──────────────────────────────
rows = []

# If we have documents, match highlights to them (doc_id ≈ book_id)
if all_documents:
    doc_dict = {doc["id"]: doc for doc in all_documents}  # Quick lookup

    for doc in all_documents:
        doc_id = doc.get("id")
        title = doc.get("title", "(no title)")
        source = doc.get("source", "unknown")
        url = doc.get("source_url") or doc.get("site_url") or ""
        updated_str = doc["updated_at"][:19].replace("T", " ") if doc.get("updated_at") else "N/A"
        doc_notes = doc.get("notes", "").strip()

        # Find matching highlights
        highlights_for_doc = [h for h in all_highlights if str(h.get("book_id")) == str(doc_id)]

        # Add doc row if notes or highlights
        if doc_notes or highlights_for_doc:
            rows.append({
                "Type": "Document",
                "Title": title,
                "URL": url,
                "Source": source,
                "Updated": updated_str,
                "Document Notes": doc_notes,
                "Highlight Text": "",
                "Highlight Note": ""
            })

            # Add highlight rows
            for h in highlights_for_doc:
                rows.append({
                    "Type": "Highlight",
                    "Title": title,
                    "URL": url,
                    "Source": source,
                    "Updated": updated_str,
                    "Document Notes": "",
                    "Highlight Text": h["text"].strip(),
                    "Highlight Note": h.get("note", "").strip()
                })

# Always add unmatched highlights (e.g., from books or legacy sources)
unmatched_highlights = [
    h for h in all_highlights
    if all_documents and str(h.get("book_id")) not in [str(d["id"]) for d in all_documents]
]
for h in unmatched_highlights:
    title = h.get("title", "(no title)")
    source = h.get("source_type", "unknown")
    url = h.get("source_url", "") or ""
    updated_str = h.get("highlighted_at", "")[:19].replace("T", " ") if h.get("highlighted_at") else "N/A"
    rows.append({
        "Type": "Highlight (unmatched doc)",
        "Title": title,
        "URL": url,
        "Source": source,
        "Updated": updated_str,
        "Document Notes": "",
        "Highlight Text": h["text"].strip(),
        "Highlight Note": h.get("note", "").strip()
    })

# If no rows yet (edge case), add a note
if not rows:
    rows = [{"Type": "Summary", "Title": "No data", "URL": "", "Source": "", "Updated": "",
             "Document Notes": f"No documents or highlights found in the last {days_back} days.",
             "Highlight Text": "", "Highlight Note": ""}]

df = pd.DataFrame(rows)

# Ensure 'Updated' column exists and sort (handle N/A gracefully)
if 'Updated' not in df.columns:
    df['Updated'] = "N/A"
df['Updated'] = pd.to_datetime(df['Updated'], errors='coerce')  # Make sortable
df = df.sort_values("Updated", ascending=False)

# Reset index after sort
df = df.reset_index(drop=True)

# ──────────────────────────────
# Part D: Display & Download
# ──────────────────────────────
total_rows = len(df)
doc_count = len([r for r in rows if r["Type"] == "Document"])
hl_count = len([r for r in rows if "Highlight" in r["Type"]])

display(Markdown(f"### Complete export: {total_rows} rows ({doc_count} documents, {hl_count} highlights)"))
display(df.head(20))  # Preview first 20 rows

# Save to CSV
filename = f"readwise_last_{days_back}_days_{datetime.now().strftime('%Y-%m-%d')}.csv"
df.to_csv(filename, index=False)
print(f"\nSaved to {filename}")

# Optional: Markdown table
md_filename = filename.replace(".csv", ".md")
with open(md_filename, "w") as f:
    f.write(df.to_markdown(index=False))
print(f"Also saved as {md_filename}")

# Download (uncomment if needed)
# from google.colab import files
# files.download(filename)

In [ ]:
pip install anthropic

In [ ]:
# ╔═══════════════════════════════════════════════════════════════╗
# ║   ONE-CLICK WEEKLY DIGEST → Claude (automatic)                ║
# ╚═══════════════════════════════════════════════════════════════╝

import anthropic
import os
from datetime import datetime
from google.colab import userdata  # ← safest way to store secrets in Colab
from google.colab import files # Import files for downloading

# ←←← PUT YOUR ANTHROPIC API KEY IN COLAB SECRETS (Sidebar → 🔑) AS "ANTHROPIC_API_KEY"
try:
    ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
except:
    ANTHROPIC_API_KEY = ""  # fallback — will ask you if missing

if not ANTHROPIC_API_KEY:
    ANTHROPIC_API_KEY = input("Paste your Anthropic API key → ").strip()

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

# Read the enhanced CSV we just created
csv_content = open(filename, "r", encoding="utf-8").read()

prompt = f"""You are my personal AI research assistant.

Below is a CSV export of everything I highlighted and noted in Readwise over the last 7 days (articles, PDFs, tweets, etc.).

Please read the entire CSV and produce a beautiful, concise weekly research digest with this exact structure:

## Theme 1 – Catchy Title
One-paragraph summary in your own words + the 1–3 most important concrete numbers/findings/quotes.

## Theme 2 – Catchy Title
(repeat)

End with a single **Through-line** paragraph that ties everything together (the big picture I seem to be tracking this week).

Use the same polished, insightful style as the digest you wrote about Anthropic productivity gains, Harvey interviews, hallucination KPIs, etc.

Here is the CSV:

{csv_content}

Thank you."""

print("Sending to Claude → this takes ~15–30 seconds...")
message = client.messages.create(
    model="claude-3-opus-20240229",   # Changed to a widely available model
    max_tokens=2000,
    temperature=0.7,
    messages=[{"role": "user", "content": prompt}]
)

digest_md = message.content[0].text

# Save + download
today = datetime.now().strftime("%Y-%m-%d")
digest_filename = f"Weekly_Digest_{today}.md"
with open(digest_filename, "w", encoding="utf-8") as f:
    f.write(f"# Readwise Weekly Digest – Week ending {today}\n\n")
    f.write(digest_md)

display(Markdown("### Here is your digest:"))
display(Markdown(digest_md))

print(f"\nSaved as {digest_filename}")
files.download(digest_filename)

Sending to Claude → this takes ~15–30 seconds...


### Here is your digest:

# Your Weekly Research Digest

## Theme 1 – The Productivity Dividend Is Here (and It's Measurable)

Anthropic's new Economic Index research moves beyond "where is AI used?" to "how much time does it actually save?" By analyzing 100,000 real Claude conversations, they found tasks that would take ~90 minutes without AI are sped up by roughly **80%**, with wide variation by profession. Extrapolating to the full economy, they estimate current-generation AI could boost annual U.S. labor productivity growth by **1.8 percentage points** over the next decade—a doubling of baseline trends that places their estimate at the high end of recent studies. The method is novel: Claude itself estimates task duration (with promising accuracy), then those micro-savings are aggregated into macro impact. It's a first attempt to quantify the "AI dividend" at scale, and the headline number—1.8%—suggests we're already past the threshold where AI moves the national productivity needle.

## Theme 2 – Collaboration Ability Is Distinct from Solo Ability

A Bayesian Item Response Theory study of 667 humans working with and without AI (Llama-3.1-8B and GPT-4o) reveals that your skill *with* AI (collaborative ability, κ) is statistically separate from your skill *without* it (solo ability, θ). Human–AI teams outperform either alone—**+23 percentage points with Llama, +29 pp with GPT-4o**—and the gains are largest on harder tasks and for lower-ability users (though higher-ability users still reach higher absolute performance). The kicker: **Theory of Mind predicts collaborative ability but not solo ability**. Users who write prompts that reflect perspective-taking and mental modeling of the AI extract better responses, even controlling for effort. This isn't just about knowing more facts; it's about a new cognitive skill—understanding and steering a non-human partner—that we can now measure and benchmark.

## Theme 3 – Reasoning Under the Hood (and What's Still Missing)

Two papers dissect *how* LLMs reason. The cognitive-foundations study maps 28 reasoning elements (goals, metacognition, representations, operations) across 170,000 traces from 17 models and finds models favor shallow forward chaining and over-rely on verification/backtracking, while humans use hierarchical abstraction and self-awareness. When researchers scaffold models with explicit cognitive structure—nudging them toward successful reasoning "storyboards"—performance on ill-structured problems jumps **up to 60%** (Qwen3-14B on dilemmas). Meanwhile, the autonomous-agents review and hallucination-mitigation work highlight gaps in perception, tool use, and factual grounding: multi-agent pipelines with JSON-based context transfer and novel KPIs (Factual Claim Density, Grounding References) can reduce hallucinations substantially, but proprietary model opacity limits true explainability. The through-line: we now have frameworks to diagnose *where* reasoning breaks and interventions that work—but the black box remains.

## Theme 4 – Hiring, Culture, and the Human Edge

Practical notes from the field: Harvey's CEO interviews candidates in Google Docs to see real collaboration and writing skill, cutting through smooth talk and AI-assisted polish. OpenAI's guide emphasizes hiring for experimentation and rewriting roles as AI shifts tasks. Interface's founder—a Trinidad-born engineer—parlays his "outsider" industrial background into an AI safety startup that found **10,800 errors in 2.5 months** at a major energy firm, winning over skeptical executives and field workers alike. The Opus 4.5 "vibe check" notes it extends the horizon of what you can code by vibing—no limit found yet—but loses some creative "magic touch" in the process, favoring confident execution over second-guessing. Across these stories: *who* you hire, *how* you assess them, and *what* diverse perspectives you bring matter as much as the models you deploy.

## Theme 5 – Risks, Guardrails, and the Governance Tug-of-War

As AI grows more capable, so do the stakes. "Vibe hacking"—low-skill bad actors using AI for phishing and malicious code—prompted Anthropic to disrupt a data-theft-and-extortion scheme in August. The same month, the White House AI Czar accused Anthropic of "fear-mongering" to shape regulation. The tension is real: every capability gain widens the attack surface and sharpens the policy fight over who sets the rules. The hallucination-KPI work and cognitive-scaffolding studies are part of the solution—better measurement, transparency, and steering—but the governance question looms large.

---

**Through-line:** This week you're tracking AI's transition from prototype to production *system*—and the new disciplines that transition demands. Anthropic's 1.8% productivity estimate and the Bayesian synergy study quantify the gains and reveal that collaboration with AI is a distinct, teachable skill (Theory of Mind matters). The reasoning and agent research show we can now map cognitive gaps, design scaffolds that unlock 60% improvements, and measure hallucinations with novel KPIs—yet proprietary opacity and the lack of human-like abstraction remain hard limits. Meanwhile, hiring and culture stories (Harvey's Google Docs interviews, Interface's outsider advantage, Opus 4.5's trade-offs) remind us that *human* judgment, diversity, and adaptability are the irreducible complement to model capability. And as productivity and power grow, so does the governance tug-of-war: who benefits, who's at risk, and who decides. You're assembling a complete picture of AI at scale—measurement, mechanism, culture, and control.


Saved as Weekly_Digest_2025-11-26.md


NameError: name 'files' is not defined